Download historical daily BTC/USD prices 

In [31]:
import requests
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.environ['ALHPA_AVANTAGE_API_KEY']
url = 'https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol=BTC&market=USD&apikey={}'.format(api_key)
r = requests.get(url)
data = r.json()

In [32]:
daily_data = data['Time Series (Digital Currency Daily)']
closing_prices = []
for day in daily_data.keys():
    closing_price = round(float(daily_data[day]['4b. close (USD)']), 2)
    closing_prices.append(closing_price)

"We got {} data points".format(len(closing_prices))

'We got 1000 data points'

Split dataset into batches of 5 sequential price points

In [33]:
time_steps = 5

In [34]:
chunks = [closing_prices[x:x+time_steps] for x in range(0, len(closing_prices), time_steps)]
train_test_split_index = int(0.8 * len(chunks))

Prepare training and test data

In [35]:
training_data = chunks[0:train_test_split_index]
test_data = chunks[train_test_split_index:]
len(training_data), training_data[0], len(test_data), test_data[0]


(160,
 [23248.86, 23141.57, 23492.09, 23554.85, 23157.07],
 40,
 [22719.71, 23455.52, 23821.61, 23107.39, 22797.16])

Turn training and test data into pytorch tensors

In [36]:
import torch
import torch.nn as nn

In [37]:
train = torch.Tensor(training_data) / 10000
test = torch.Tensor(test_data) / 10000
train.shape, test.shape

(torch.Size([160, 5]), torch.Size([40, 5]))

In [38]:
input_to_output = nn.Linear(1 + time_steps, 1)
input_to_hidden = nn.Linear(1 + time_steps, time_steps)

Feed forward operation

In [39]:
def forward(input, hidden):
    combined_input = torch.cat((input, hidden), 1)
    output = input_to_output(combined_input)
    new_hidden = input_to_hidden(combined_input)
    return output, new_hidden

In [40]:
hidden = torch.zeros(1, time_steps)
for i in range(0, time_steps):
    input = train[0][i:i+1].unsqueeze(1)
    output, hidden = forward(input, hidden)
print(output)

tensor([[0.7215]], grad_fn=<AddmmBackward0>)


Train the model in randomized batches in one epoch

In [41]:
batch_size = 10
learning_rate = 0.0005
mse = nn.MSELoss()

input_to_output = nn.Linear(1 + time_steps, 1)
input_to_hidden = nn.Linear(1 + time_steps, time_steps)

def run_epoch():
    randomized_training_indices = torch.randperm(train.shape[0])
    losses = []
    # for each training batch
    for new_batch_start in range(0, len(randomized_training_indices), batch_size):
        batch_indices = [randomized_training_indices[x].item() for x in range(new_batch_start, new_batch_start+batch_size)]
        
        predictions = torch.zeros(len(batch_indices))
        outputs = torch.zeros(len(batch_indices))
        index = 0

        input_to_hidden.zero_grad()
        input_to_output.zero_grad()
    
        # for training data in batch
        for batch_index in batch_indices:

            hidden = torch.zeros(1, time_steps)
            single_train_data = train[batch_index]
            prediction = 0
            # for each time step
            for step in range(0, len(single_train_data - 1)):
                input = single_train_data[step:step+1].unsqueeze(1)
                prediction, hidden = forward(input, hidden)          
            
            actual = single_train_data[-1]
            predictions[index] = prediction
            outputs[index] = actual
            index += 1
        
        # calculate error
        loss = mse(predictions, outputs)
        losses.append(loss.item())

        # back propagation to adjust weights
        loss.backward()
        for p in input_to_hidden.parameters():
            p.data.add_(p.grad.data, alpha=-learning_rate)
        input_to_hidden.grad = None

        for p in input_to_output.parameters():
            p.data.add_(p.grad.data, alpha=-learning_rate)
        input_to_output.grad = None
    
    print("avg epoch loss: {}".format(sum(losses)/ len(losses)))

In [42]:
for i in range(0, 30):
    run_epoch()

avg epoch loss: 4.754158481955528
avg epoch loss: 1.7350148484110832
avg epoch loss: 0.5397066846489906
avg epoch loss: 0.14634726475924253
avg epoch loss: 0.03944894589949399
avg epoch loss: 0.014219910284737125
avg epoch loss: 0.008661530824610963
avg epoch loss: 0.007340395750361495
avg epoch loss: 0.007011319074081257
avg epoch loss: 0.006899345695273951
avg epoch loss: 0.006823368443292566
avg epoch loss: 0.006763444063835777
avg epoch loss: 0.006699280478642322
avg epoch loss: 0.006637948579736985
avg epoch loss: 0.006563397793797776
avg epoch loss: 0.006511490239063278
avg epoch loss: 0.0064420787239214405
avg epoch loss: 0.006379861995810643
avg epoch loss: 0.006335714380838908
avg epoch loss: 0.006272993370657787
avg epoch loss: 0.006227822159416974
avg epoch loss: 0.0061538691516034305
avg epoch loss: 0.006098491765442304
avg epoch loss: 0.006035747574060224
avg epoch loss: 0.005993629529257305
avg epoch loss: 0.0059376349818194285
avg epoch loss: 0.005876464259927161
avg epo

Check prediction on test data

In [43]:
for i in range(0, test.shape[0]):
    single_test_data = test[i]
    print(single_test_data)
    hidden = torch.zeros(1, time_steps)
    prediction = 0
    # for each time step
    for step in range(0, len(single_test_data) - 1):
        input = single_test_data[step:step+1].unsqueeze(1)
        prediction, hidden = forward(input, hidden)          
    
    actual = single_test_data[-1]
    percentage_error = round(((prediction.item() - actual.item()) * 100) / actual.item(), 3)
    print("prediction: {}, actual: {}, percentage error={}".format(prediction  * 10000, actual * 10000, percentage_error))

tensor([2.2720, 2.3456, 2.3822, 2.3107, 2.2797])
prediction: tensor([[22997.9258]], grad_fn=<MulBackward0>), actual: 22797.16015625, percentage error=0.881
tensor([2.1336, 1.9426, 1.9273, 1.9175, 1.8809])
prediction: tensor([[19178.1816]], grad_fn=<MulBackward0>), actual: 18808.689453125, percentage error=1.964
tensor([1.8037, 1.8255, 1.8541, 1.8324, 1.9167])
prediction: tensor([[18345.8672]], grad_fn=<MulBackward0>), actual: 19166.900390625, percentage error=-4.284
tensor([1.9359, 1.9148, 1.8651, 1.9422, 1.9204])
prediction: tensor([[19207.9961]], grad_fn=<MulBackward0>), actual: 19204.08984375, percentage error=0.02
tensor([1.8765, 1.9696, 1.8185, 1.7720, 1.7140])
prediction: tensor([[17946.1797]], grad_fn=<MulBackward0>), actual: 17139.51953125, percentage error=4.706
tensor([1.7149, 1.8719, 1.9160, 1.8368, 1.8414])
prediction: tensor([[18501.1348]], grad_fn=<MulBackward0>), actual: 18414.4296875, percentage error=0.471
tensor([1.8704, 1.8656, 1.7803, 1.7776, 1.7659])
prediction: te